In [1]:
import pandas as pd
import numpy as np
#from sqlalchemy import create_engine
#import psycopg2
#from config import db_password,g_key
from config import resolution,lat_max,lat_min,lng_max,lng_min
from config import t_sat,flow_rate,project_span
import json
#from shapely.geometry import Point,shape
import os
from datetime import datetime,timedelta,date
from functions import Rmax_calc,recovery_calc,month_index
import scipy.optimize
from geojson import Feature, FeatureCollection, Polygon

In [2]:
# blocks information
file_path = os.path.join('..','..','Resources','blocks_info.csv')
blocks_df = pd.read_csv(file_path)
# actual gold produciton by month
file_path = os.path.join('..','..','Resources','act_monthly_oz.csv')
act_oz_df = pd.read_csv(file_path)

In [3]:
# Convert to datetime format
blocks_df['stack_finish'] = pd.to_datetime(blocks_df['stack_finish'])
blocks_df['leach_start'] = pd.to_datetime(blocks_df['leach_start'])
blocks_df['leach_end'] = pd.to_datetime(blocks_df['leach_end'])
blocks_df['leach_days'] = blocks_df['leach_end']-blocks_df['leach_start']
lat_lng_list = blocks_df[['lat','lng']].values.tolist()
# Create a dataframe with sequence as block_id, lift and lat_lng as columns
block_leach_time = pd.DataFrame(blocks_df[['lift']])
block_leach_time['lat_lng']=blocks_df[['lat','lng']].values.tolist()
block_leach_time.index.name = 'block_id'
# Create block_list for later index reference.
block_list = block_leach_time.index.tolist()

# Set up project span
projectspan = (
    datetime.strptime(project_span['startdate'],'%Y-%m-%d'),
    datetime.strptime(project_span['enddate'],'%Y-%m-%d')
    )
# Create month list of the project
startyear = projectspan[0].year
startmonth = projectspan[0].month
endyear = projectspan[1].year
endmonth = projectspan[1].month
monthlist = [datetime(m//12, m%12+1, 1) for m in range(startyear*12+startmonth-1, endyear*12+endmonth)]
# Use monthlist create recovery dataframe
leach_days_df = pd.DataFrame(monthlist,columns = ['months'])
leach_ks_df = pd.DataFrame(monthlist,columns = ['months'])
leach_rec_df = pd.DataFrame(monthlist,columns = ['months'])
leach_ozs_df = pd.DataFrame(monthlist,columns = ['months'])

# Break dataframe to records
blocks_records = blocks_df.to_records(index=False)

In [4]:
# This block of code calculates a specific block's recovery over months.
## The function takes two block indices 
## and the k factor that is going to be used for recovery calculation.
def block_leach_days(i,ref,lift_diff):
    # if the two indices are the same, means this is calculating the primary leach recovery
    calc_block = blocks_records[i]
    if i == ref:
        start_date = pd.to_datetime(calc_block['leach_start'])
        end_date = pd.to_datetime(calc_block['leach_end'])
    # if the two indices are different, means calculating recoveries of blocks under secondary leach
    else:
        # ref_block is the block on the very top
        ref_block = blocks_records[ref]
        # the blocks below the ref_block inherit the leaching dates from the ref_block
        # calculate leach delay
        start_date = pd.to_datetime(ref_block['leach_start']) + timedelta(days = t_sat + round(lift_diff * flow_rate))
        end_date = pd.to_datetime(ref_block['leach_end']) + timedelta(days = round(lift_diff * flow_rate))
    # locate the month row
    m_index = month_index(start_date,projectspan)
    start_month = start_date.month
    end_month = end_date.month + (end_date.year - start_date.year) * 12
    # calcuate how much recovery that this block has achieved since the start
    for m in range(start_month,end_month+1):
        # calculate how many days in this month for recovery calculation
        if m == start_month:
            if m == 12:
                days = 31 - start_date.day
            else:
                days = (datetime(start_date.year,start_date.month+1,1)-start_date).days
        elif m == end_month:
            days = end_date.day
        elif m < 12:
            days = (datetime(start_date.year,m+1,1)-datetime(start_date.year,m,1)).days
        elif m == 12:
            days = 31
        else:
            days = (datetime(start_date.year+1,m-12+1,1)-datetime(start_date.year+1,m-12,1)).days
        # write the recovery to the rec_df.
        leach_days_df.at[m_index,i] = days
        m_index += 1

In [5]:
def create_leach_days_df():
    for high in block_list:
        leach_days_df[high]=0
        lift_diff = 0
        block_leach_days(high,high,lift_diff)
        high_lift = blocks_records[high]['lift']
        for low in range(0,high):
            if lat_lng_list[high]==lat_lng_list[low]:
                low_lift = blocks_records[low]['lift']
                lift_diff = high_lift-low_lift
                block_leach_days(low,high,lift_diff)

In [6]:
# Have no idea why this function has to be ran twice to make the records the same as the dataframe
create_leach_days_df()
create_leach_days_df()

In [7]:
print(leach_days_df[1000].tolist())
leach_days_records = leach_days_df.to_records()
print(leach_days_records['1000'].tolist())

[0, 0, 0, 0, 0, 3, 31, 29, 0, 0, 0, 23, 31, 29, 6, 15, 31, 30, 31, 11, 28, 31, 30, 29, 0, 0, 0, 10, 31, 30, 29, 0, 23, 27, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 3, 31, 29, 0, 0, 0, 23, 31, 29, 6, 15, 31, 30, 31, 11, 28, 31, 30, 29, 0, 0, 0, 10, 31, 30, 29, 0, 23, 27, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [8]:
# Prepare k factors
def k_calc(k):
    for b in block_list:
        leach_ks_df[b] = k
        leach_rec_df[b] = 0.0
    return leach_ks_df,leach_rec_df
leach_ks_df,leach_rec_df = k_calc(0.005)
# leach_ozs_df = leach_rec_df.copy()
# Prepare Rmax
rmax = blocks_df['rmax']
cont_ozs = blocks_df['ounces_per_block']

In [9]:
leach_ks_records = leach_ks_df.to_records(index=False)
leach_days_records = leach_days_df.to_records(index=False)
leach_rec_records = leach_rec_df.to_records(index=False)
#leach_ozs_records = leach_ozs_df.to_records(index=False)

In [10]:
def rec_monthly(k):
    for b in block_list:
        rcum = 0
        b_rec = leach_rec_records[f'{b}']
        for m in range(0,len(monthlist)):
            # b_rec[m] = recovery_calc(rmax[b],rcum,leach_ks_records[f'{b}'][m],leach_days_records[f'{b}'][m])
            b_rec[m] = recovery_calc(rmax[b],rcum,k,leach_days_records[f'{b}'][m])
            rcum += b_rec[m]
        leach_ozs_df[b] = b_rec * cont_ozs[b]
        #leach_rec_records[f'{b}'] = b_rec
    model_monthly = leach_ozs_df.sum(axis=1)
    #leach_ozs_df['oz_monthly'] = model_monthly
    return model_monthly

# Optimize K value to make model close to actual

In [11]:
current_date = date.today()
opt_till = month_index(current_date,projectspan)

In [12]:
def sq(k):
    # use this k to calculate gold production
    model_rec = rec_monthly(k)[:opt_till]
    actual_rec = act_oz_df.iloc[:opt_till]['ounces']
    delta = np.nansum((actual_rec-model_rec)**2)
    # delta = abs(calculated recovery - actual recovery)
    return delta

In [13]:
initial_k = 0.001

In [14]:
# Check other options and see steps.
opt1 = scipy.optimize.minimize(sq,initial_k,method='BFGS',tol=100)
print(f'opt 1 error is {opt1.fun}, when k is {opt1.x[0]}')
opt2 = scipy.optimize.minimize(sq,initial_k,method='Nelder-Mead',tol=100,bounds=[(0.000001,0.020001)])
print(f'opt 2 error is {opt2.fun}, when k is {opt2.x[0]}')
if opt1.fun > opt2.fun:
    k_to_use = opt2.x[0]
    print(f'Chose to use opt 2')
else:
    k_to_use = opt1.x[0]
    print(f'Chose to use opt 1')

opt 1 error is 2.3585828331520544e-05, when k is 0.005999992541362125


C:\Users\dzhang\Anaconda3\lib\site-packages\scipy\optimize\_minimize.py:535: RuntimeWarning: Method Nelder-Mead cannot handle constraints nor bounds.
  warn('Method %s cannot handle constraints nor bounds.' % method,


opt 2 error is 1052.4129341931457, when k is 0.006050000000000018
Chose to use opt 1


# Prepare Data For Visualization

In [15]:
monthly_oz = rec_monthly(k_to_use)
leach_ozs_df['oz_monthly'] = monthly_oz

In [16]:
# This function is to calculate the remaining ounces of blocks without leaching
def init_oz(X):
    # input X is the dataframe's row in a series
    m_place = month_index(X[-2],projectspan)
    Y = X.copy()
    Y[m_place + 2:-2] = Y[-1]
    return Y

In [17]:
# Creating cells gold production stats by month.
blocks_oz_df = leach_ozs_df.set_index('months').T

## Export the data for visualization
file_path = os.path.join('..','..','Resources','blocks_oz.json')
blocks_oz_df.to_json(file_path)
blocks_oz_df.drop(blocks_oz_df.tail(1).index,inplace=True)

## Insert cell column for groupby.
blocks_oz_df.insert(0,'cell',blocks_df['cell'])
cell_oz_df = blocks_oz_df.groupby(['cell']).sum()

## Export the data for visualization
file_path = os.path.join('..','..','Resources','cells_oz.json')
cell_oz_df.to_json(file_path)

# Create gold production by position
blocks_oz_df.insert(0,"lat",blocks_df['lat'])
blocks_oz_df.insert(1,"lng",blocks_df['lng'])
position_oz_df = blocks_oz_df
position_oz_df.drop(columns = 'cell',inplace = True)
position_oz_df = position_oz_df.groupby(['lat','lng']).sum()
position_oz_df.reset_index(inplace = True)

# Blocks cumulative gold production
cumsumlist = blocks_oz_df.columns.tolist()[2:]
blocks_cumsum_oz_df = blocks_oz_df.copy()
blocks_cumsum_oz_df[cumsumlist] = blocks_cumsum_oz_df[cumsumlist].cumsum(axis = 1)

# Blocks remaining ounces
blocks_remaining_oz_df = blocks_df[['lat','lng']]
blocks_remaining_oz_df[cumsumlist] = 0.0
blocks_remaining_oz_df[['stack_finish','ounces_per_block']] = blocks_df[['stack_finish','ounces_per_block']]
blocks_remaining_oz_df = blocks_remaining_oz_df.apply(init_oz,axis=1)
blocks_remaining_oz_df.drop(columns = ['stack_finish','ounces_per_block'],inplace = True)
blocks_remaining_oz_df[cumsumlist]=blocks_remaining_oz_df[cumsumlist]-blocks_cumsum_oz_df[cumsumlist]

# Position remaining ounces
position_remaining_df = blocks_remaining_oz_df.groupby(['lat','lng']).sum()
position_remaining_df.reset_index(inplace = True)

# Convert datetime columns to string
date_list = position_remaining_df.columns.tolist()[2:]
converted_date_list = [str(date.year)+'-'+str(date.month) if date.month>=10 else str(date.year)+'-0'+str(date.month) for date in date_list]
convert_dict = dict(zip(date_list, converted_date_list))
position_remaining_df.rename(columns = convert_dict,inplace = True)

<ipython-input-17-57e3d9d9cc88>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blocks_remaining_oz_df[cumsumlist] = 0.0
C:\Users\dzhang\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\dzhang\Anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [18]:
# Geojson
# Define block size
delta_lat = (lat_max - lat_min)/resolution[0]/2
delta_lng = (lng_max - lng_min)/resolution[1]/2
features = position_remaining_df.apply(
    lambda row: Feature(geometry=Polygon([[
        # (row['lat'] + delta_lat, row['lng'] + delta_lng),
        # (row['lat'] - delta_lat, row['lng'] + delta_lng),
        # (row['lat'] - delta_lat, row['lng'] - delta_lng),
        # (row['lat'] + delta_lat, row['lng'] - delta_lng),
        # (row['lat'] + delta_lat, row['lng'] + delta_lng)
        (row['lng'] + delta_lng, row['lat'] + delta_lat),
        (row['lng'] - delta_lng, row['lat'] + delta_lat),
        (row['lng'] - delta_lng, row['lat'] - delta_lat),
        (row['lng'] + delta_lng, row['lat'] - delta_lat),
        (row['lng'] + delta_lng, row['lat'] + delta_lat)
    ]])),
    axis=1).tolist()

properties = position_remaining_df.drop(['lat', 'lng'], axis=1).to_dict('records')

for x in range(len(features)):
    features[x].properties = properties[x]

feature_collection = FeatureCollection(features=features)

file_path = os.path.join('..','..','Resources','position_remain_oz.geojson')

with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(feature_collection, f, ensure_ascii=False)

position_oz_df['overall_ozs'] = position_oz_df.drop(columns = ['lat','lng']).sum(axis = 1)
file_path = os.path.join('..','..','Resources','latlng_oz.json')
position_oz_df.to_json(file_path)